In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_csv(r"C:\Users\hridy\Documents\Innomatics\Sentiment Analysis\reviews_data_dump\reviews_badminton\data.csv")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hridy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hridy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
X = df['Review text']
y = df['Ratings']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

print(X_train.shape, X_test.shape)


(6388,) (2130,)


In [3]:
import mlflow

mlflow.set_experiment("Sentiment_Analysis_Hridya")

2024/03/29 08:21:03 INFO mlflow.tracking.fluent: Experiment with name 'Sentiment_Analysis_Hridya' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/hridy/Documents/Innomatics/SentimentAnalysis_app/mlruns/779996273720896002', creation_time=1711680663110, experiment_id='779996273720896002', last_update_time=1711680663110, lifecycle_stage='active', name='Sentiment_Analysis_Hridya', tags={}>

In [5]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_csv(r"C:\Users\hridy\Documents\Innomatics\Sentiment Analysis\reviews_data_dump\reviews_badminton\data.csv")

def preprocess_text(text):
    if pd.notnull(text):
        text = str(text).lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        tokens = text.split()
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        clean_text = ' '.join(tokens)
        return clean_text
    else:
        return ''

df['Cleaned Text'] = df['Review text'].apply(preprocess_text)

threshold = 3

df['Sentiment'] = df['Ratings'].apply(lambda x: 1 if x >= threshold else 0)

X_train, X_test, y_train, y_test = train_test_split(df['Cleaned Text'], df['Sentiment'], test_size=0.2, random_state=42)

models = [
    ('Random Forest', RandomForestClassifier()),
    ('Logistic Regression', LogisticRegression()),
    ('Support Vector Machine', SVC())
]

pipelines = []
for model_name, model in models:
    pipeline = Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('model', model)
    ])
    pipelines.append((model_name, pipeline))


mlflow.sklearn.autolog(max_tuning_runs=None)

param_grid = {
    'Random Forest': {
        'model__n_estimators': [100, 200, 300],
        'model__max_depth': [None, 10, 20]
    },
    'Logistic Regression': {
        'model__C': [0.1, 1.0, 10.0],
        'model__solver': ['liblinear', 'lbfgs']
    },
    'Support Vector Machine': {
        'model__C': [0.1, 1.0, 10.0],
        'model__kernel': ['linear', 'rbf']
    }
}

grid_search_results = {}
for model_name, pipeline in pipelines:
    param_grid_model = param_grid[model_name]
    clf = GridSearchCV(pipeline, param_grid=param_grid_model, scoring='f1_weighted', cv=5, return_train_score=True, verbose=1)
    with mlflow.start_run() as run:
        %time clf.fit(X_train, y_train)
    grid_search_results[model_name] = clf

for model_name, grid_search in grid_search_results.items():
    print(f"Best hyperparameters for {model_name}:")
    print(grid_search.best_params_)
    print("Best F1-score:", grid_search.best_score_)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hridy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hridy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2024/03/29 08:21:37 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 9 candidates, totalling 45 fits


2024/03/29 08:29:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Python311\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."
2024/03/29 08:29:49 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2024/03/29 08:29:53 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2024/03/29 08:29:54 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


CPU times: total: 2min 42s
Wall time: 8min 16s
Fitting 5 folds for each of 6 candidates, totalling 30 fits


c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

CPU times: total: 2.92 s
Wall time: 12.9 s
Fitting 5 folds for each of 6 candidates, totalling 30 fits


2024/03/29 08:30:51 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2024/03/29 08:30:55 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


CPU times: total: 19.3 s
Wall time: 48.9 s
Best hyperparameters for Random Forest:
{'model__max_depth': None, 'model__n_estimators': 300}
Best F1-score: 0.8969918318799899
Best hyperparameters for Logistic Regression:
{'model__C': 10.0, 'model__solver': 'liblinear'}
Best F1-score: 0.9129151614171065
Best hyperparameters for Support Vector Machine:
{'model__C': 1.0, 'model__kernel': 'linear'}
Best F1-score: 0.906410623818228


In [6]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_csv(r"C:\Users\hridy\Documents\Innomatics\Sentiment Analysis\reviews_data_dump\reviews_badminton\data.csv")

def preprocess_text(text):
    if pd.notnull(text):
        text = str(text).lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        tokens = text.split()
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        clean_text = ' '.join(tokens)
        return clean_text
    else:
        return ''

df['Cleaned Text'] = df['Review text'].apply(preprocess_text)

threshold = 3

df['Sentiment'] = df['Ratings'].apply(lambda x: 1 if x >= threshold else 0)

X_train, X_test, y_train, y_test = train_test_split(df['Cleaned Text'], df['Sentiment'], test_size=0.2, random_state=42)

mlflow.sklearn.autolog(max_tuning_runs=None)

param_grid_svm = {
    'model__C': [0.1, 1.0, 10.0],
    'model__kernel': ['linear', 'rbf']
}

svm_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model', SVC())
])

clf_svm = GridSearchCV(svm_pipeline, param_grid=param_grid_svm, scoring='f1_weighted', cv=5, return_train_score=True, verbose=1)
with mlflow.start_run() as run:
    clf_svm.fit(X_train, y_train)

print("Best hyperparameters for Support Vector Machine:")
print(clf_svm.best_params_)
print("Best F1-score:", clf_svm.best_score_)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hridy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hridy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2024/03/29 08:32:51 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 6 candidates, totalling 30 fits


2024/03/29 08:33:58 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2024/03/29 08:34:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Best hyperparameters for Support Vector Machine:
{'model__C': 1.0, 'model__kernel': 'linear'}
Best F1-score: 0.906410623818228


In [12]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Define pipelines
pipelines = {
    'random_forest': Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('scaler', StandardScaler(with_mean=False)),  # Set with_mean=False for sparse matrices
        ('classifier', RandomForestClassifier())
    ]),
    'logistic_regression': Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('scaler', StandardScaler(with_mean=False)),  # Set with_mean=False for sparse matrices
        ('classifier', LogisticRegression())
    ]),
    'svc': Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('scaler', StandardScaler(with_mean=False)),  # Set with_mean=False for sparse matrices
        ('classifier', SVC())
    ])
}

# Define parameter grid for each algorithm
param_grids = {
    'random_forest': {
        'vectorizer__ngram_range': [(1, 1), (1, 2)],
        'classifier__n_estimators': [50, 100, 200]
    },
    'logistic_regression': {
        'vectorizer__ngram_range': [(1, 1), (1, 2)],
        'classifier__C': [0.1, 1, 10],
        'classifier__penalty': ['l1', 'l2']
    },
    'svc': {
        'vectorizer__ngram_range': [(1, 1), (1, 2)],
        'classifier__C': [0.1, 1, 10],
        'classifier__kernel': ['linear', 'rbf']
    }
}

best_models = {}

# Run the Pipeline
for algo in pipelines.keys():
    print("*"*10, algo, "*"*10)
    
    # Create GridSearchCV
    grid_search = GridSearchCV(estimator=pipelines[algo], 
                               param_grid=param_grids[algo], 
                               cv=5, 
                               scoring='accuracy', 
                               return_train_score=True,
                               verbose=1)
    
    # Enable autologging
    mlflow.sklearn.autolog(max_tuning_runs=None)
    
    # Start MLflow run
    # Start MLflow run
    with mlflow.start_run() as run:
        %time grid_search.fit(X_train, y_train)

        # Print results
    print('Train Score: ', grid_search.best_score_)
    print('Test Score: ', grid_search.best_estimator_.score(X_test, y_test))
    
    # Store best model
    best_models[algo] = grid_search.best_estimator_
    print()


2024/03/29 10:16:22 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


********** random_forest **********
Fitting 5 folds for each of 6 candidates, totalling 30 fits


2024/03/29 10:23:40 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2024/03/29 10:23:43 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


CPU times: total: 2min 40s
Wall time: 7min 21s
Train Score:  0.9028473297328278
Test Score:  0.9061032863849765

********** logistic_regression **********


2024/03/29 10:23:44 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


...

CPU times: total: 3.86 s
Wall time: 18.8 s
Train Score:  0.910626231546332
Test Score:  0.914906103286385

********** svc **********


2024/03/29 10:24:04 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 12 candidates, totalling 60 fits


2024/03/29 10:31:33 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2024/03/29 10:31:36 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


CPU times: total: 3min 10s
Wall time: 7min 33s
Train Score:  0.9076916364200505
Test Score:  0.9166666666666666



In [ ]:
# Close MLflow run
mlflow.end_run()

In [ ]:
mlflow.sklearn.autolog(disable=True)